In [ ]:
%%capture

#!pip install numpy==1.17.5
#!pip install -U numpy==1.11.0
!pip3 install numpy==1.17.4

import numpy as np
import pandas as pd 
from tqdm import tqdm

from matplotlib import image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import cv2

!pip install fastai --upgrade

import torch 
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

from fastai.vision.all import *

In [ ]:
!wget https://drive.google.com/u/0/open?id=1a2oHjcEcwXP8oUF95qiwrqzACb2YlUhn

Sequences from day 1

In [ ]:
plt_image = image.imread('../input/lisa-traffic-light-dataset/daySequence1/daySequence1/frames/daySequence1--00000.jpg')
print(plt_image.dtype)
print(plt_image.shape)
plt.imshow(plt_image)
plt.show()

In [ ]:
plt_image = image.imread('../input/lisa-traffic-light-dataset/daySequence1/daySequence1/frames/daySequence1--00900.jpg')
print(plt_image.dtype)
print(plt_image.shape)
plt.imshow(plt_image)
plt.show()

In [ ]:
plt_image = image.imread('../input/lisa-traffic-light-dataset/nightSequence1/nightSequence1/frames/nightSequence1--00000.jpg')
print(plt_image.dtype)
print(plt_image.shape)
plt.imshow(plt_image)
plt.show()

In [ ]:
plt_image = image.imread('../input/lisa-traffic-light-dataset/nightSequence1/nightSequence1/frames/nightSequence1--00100.jpg')
print(plt_image.dtype)
print(plt_image.shape)
plt.imshow(plt_image)
plt.show()

In [ ]:
day1_frame_annotations_BOX = pd.read_csv("../input/lisa-traffic-light-dataset/Annotations/Annotations/daySequence1/frameAnnotationsBOX.csv", sep=';')
day1_frame_annotations_BULB = pd.read_csv("../input/lisa-traffic-light-dataset/Annotations/Annotations/daySequence1/frameAnnotationsBULB.csv", sep=';')
night1_frame_annotations_BOX = pd.read_csv("../input/lisa-traffic-light-dataset/Annotations/Annotations/nightSequence1/frameAnnotationsBOX.csv", sep=';')
night1_frame_annotations_BULB = pd.read_csv("../input/lisa-traffic-light-dataset/Annotations/Annotations/nightSequence1/frameAnnotationsBULB.csv", sep=';')
day2_frame_annotations_BOX = pd.read_csv("../input/lisa-traffic-light-dataset/Annotations/Annotations/daySequence2/frameAnnotationsBOX.csv", sep=';')
day2_frame_annotations_BULB = pd.read_csv("../input/lisa-traffic-light-dataset/Annotations/Annotations/daySequence2/frameAnnotationsBULB.csv", sep=';')

day2_frame_annotations_BOX.head(3)

In [ ]:
#frame_annotations_BULB.iloc[0]['']

plt_image = image.imread('../input/lisa-traffic-light-dataset/daySequence1/daySequence1/frames/daySequence1--00000.jpg')

fig, ax = plt.subplots()

ax.imshow(plt_image)

# Create a Rectangle patch
rect = patches.Rectangle((710, 481), 714-710, 486-481, linewidth=1, edgecolor='r', facecolor='none')

ax.add_patch(rect)

plt.show()

## Data Pipeline

In [ ]:
def center_format(df):
    cols = ['name', 'center_x', 'center_y', 'height', 'width', 'tag']
    tags = {'stopLeft': 0, 'go': 1, 'warningLeft': 2, 'stop': 0, 'warning': 2, 'goLeft': 1, 'goForward': 1}
    tags_idx = 0

    boxes_df = pd.DataFrame(columns=cols)
    for row in tqdm(df.iterrows()):
        center_x = (row[1]['Lower right corner X'] + row[1]['Upper left corner X'])/2
        center_y = (row[1]['Lower right corner Y'] + row[1]['Upper left corner Y'])/2
        height = row[1]['Lower right corner Y'] - row[1]['Upper left corner Y']
        width = row[1]['Lower right corner X'] - row[1]['Upper left corner X']
        tag = tags[row[1]['Annotation tag']]
    
        name = row[1]['Filename']
        data_dict = {'name': name, 'center_x': center_x , 'center_y': center_y, 'height': height, 'width': width, 'tag': tag}
        boxes_df.loc[row[0]] = pd.Series(data_dict)
        
    return boxes_df


def min_max_box_format(df):
    cols = ['name', 'x_min', 'y_min', 'x_max', 'y_max', 'class_id']
    tags = {'stopLeft': 0, 'go': 1, 'warningLeft': 2, 'stop': 0, 'warning': 2, 'goLeft': 1, 'goForward': 1}
    tags_idx = 0

    boxes_df = pd.DataFrame(columns=cols)
    for row in tqdm(df.iterrows()):
        min_x = row[1]['Upper left corner X']
        min_y = row[1]['Upper left corner Y'] 
        max_y = row[1]['Lower right corner Y']
        max_x = row[1]['Lower right corner X'] 
        tag = tags[row[1]['Annotation tag']]
        name = row[1]['Filename']
        data_dict = {'name': name, 'x_min': min_x , 'y_min': min_y, 'x_max': max_x, 'y_max': max_y, 'class_id': tag}
        boxes_df.loc[row[0]] = pd.Series(data_dict)
    return boxes_df

In [ ]:
#min_max_boxes_df_day1_BULB = min_max_box_format(day1_frame_annotations_BULB)
min_max_boxes_df_day1_BOX = min_max_box_format(day1_frame_annotations_BOX)
#min_max_boxes_df_night1_BULB = min_max_box_format(night1_frame_annotations_BULB)
min_max_boxes_df_day2_BOX = min_max_box_format(day2_frame_annotations_BOX)
min_max_boxes_df_day2_BOX

In [ ]:
num_classes = len(np.unique(min_max_boxes_df_day2_BOX['class_id'])) + 1 # 3 classes (Stop, Warning, Go) + Background
num_classes

In [ ]:
import albumentations

In [ ]:
def resize_image(img_arr, bboxes, h, w):
    """
    :param img_arr: original image as a numpy array
    :param bboxes: bboxes as numpy array where each row is 'x_min', 'y_min', 'x_max', 'y_max', "class_id"
    :param h: resized height dimension of image
    :param w: resized weight dimension of image
    :return: dictionary containing {image:transformed, bboxes:['x_min', 'y_min', 'x_max', 'y_max', "class_id"]}
    """
    # create resize transform pipeline
    transform = albumentations.Compose(
        [albumentations.Resize(height=h, width=w, always_apply=True)],
        bbox_params=albumentations.BboxParams(format='pascal_voc'))

    transformed = transform(image=img_arr, bboxes=bboxes)

    return transformed

In [ ]:
bbox_cols = ['x_min', 'y_min', 'x_max', 'y_max']

def get_target_ds(name, df):  
    rows = df[df["name"] == 'dayTest/'+name[-23:]]  
    bbox = []
    for row, class_id in zip(rows[bbox_cols].values, rows["class_id"].values):
        bbox_entry = []
        for box in row:
            bbox_entry.append(box)
        bbox_entry.append(class_id)
        bbox.append(bbox_entry)
    return bbox

class LisaDataset(torch.utils.data.Dataset):  
    def __init__(self, images_path, df, std=False):
        super(LisaDataset, self).__init__()
        self.images_path = images_path
        self.df = df
        self.std = std
        
    def __len__(self):
        return len(self.images_path)  

    def __getitem__(self, idx):
        img_path = self.images_path[idx]
        img = cv2.imread(str(img_path), cv2.IMREAD_UNCHANGED)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        
        boxes = get_target_ds(img_path.name, self.df)
            
        transformed_dict = resize_image(img, boxes, 256, 256)

        # contains the image as array
        img = transformed_dict["image"]

        # contains the resized bounding boxes
        boxes_with_labels = np.array(list(map(list, transformed_dict["bboxes"]))).astype(float)
        
        if self.std:
            img = img/255.0
            
        img = img.reshape(3, 256, 256)
        #img = cv2.resize(img, (512, 512))
        target = {}
        
        if len(boxes_with_labels.shape) == 1:
            if boxes_with_labels.shape[0] == 0:
                boxes = np.array([[], [], [], []]).T
                labels = torch.as_tensor(np.array([]), dtype=torch.int64)
                areas = np.array([])
            else:
                boxes = boxes_with_labels[:-1].astype(np.int64)
                labels = torch.as_tensor(boxes_with_labels[-1].astype(np.float), dtype=torch.int64)
                areas = (boxes[3] - boxes[1]) * (boxes[2] - boxes[0])
        else:
            boxes = boxes_with_labels[:, :-1].astype(np.int64)
            labels = torch.as_tensor(boxes_with_labels[:, -1].astype(np.float), dtype=torch.int64)
            areas = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
            
        iscrowd = torch.zeros((boxes.shape[0],))
        image_id = torch.tensor([idx])
        areas = torch.as_tensor(areas.astype(np.float), dtype=torch.double)
        boxes = torch.from_numpy(boxes) 
        boxes = torch.as_tensor(boxes, dtype=torch.int64)
        
        target["boxes"] = boxes
        target["labels"] = labels
        target["area"] = areas
        target["iscrowd"] = iscrowd
        target["image_id"] = image_id    
        img = torch.from_numpy(img)
        img = torch.as_tensor(img, dtype=torch.double)
        
        return img, target

Now we read the day 1 sequence images

In [ ]:
day_sequence_1 = Path("../input/lisa-traffic-light-dataset/daySequence1/daySequence1/frames/") 
day_sequence_1_images = get_image_files(day_sequence_1)
day_sequence_2 = Path("../input/lisa-traffic-light-dataset/daySequence2/daySequence2/frames/") 
day_sequence_2_images = get_image_files(day_sequence_2)

img = cv2.imread(str(day_sequence_1_images[-1]), cv2.IMREAD_UNCHANGED)
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
plt.imshow(img)

In [ ]:
dataset = LisaDataset(day_sequence_1_images, min_max_boxes_df_day1_BOX)
img, target = dataset.__getitem__(0)
img = img.reshape(256, 256, 3)
img = torch.as_tensor(img, dtype=torch.int)
print(img.shape, target["boxes"], target["labels"])

fig, ax = plt.subplots(figsize=(20,20))
ax.imshow(img)
rect = patches.Rectangle((76, 199), 87-76, 223-199, linewidth=1, edgecolor='r', facecolor='none')
ax.add_patch(rect)
rect = patches.Rectangle((241, 166), 247-241, 180-166, linewidth=1, edgecolor='r', facecolor='none')
ax.add_patch(rect)

plt.show()

Build the Model

In [ ]:
if(torch.cuda.is_available()):
    device = torch.device("cuda")
    print("Device:", device, torch.cuda.get_device_name(0))
else:
    device= torch.device("cpu")
    print("Device:", device)

build the dataloader

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
%%sh
if [ ! -d "output/vision" ]; then
    git clone https://github.com/pytorch/vision.git output/vision
else
    echo "output/vision already cloned"
fi

cp output/vision/references/detection/utils.py .
cp output/vision/references/detection/transforms.py .
cp output/vision/references/detection/coco_eval.py .
cp output/vision/references/detection/engine.py .
cp output/vision/references/detection/coco_utils.py .

In [ ]:
%%capture
!pip3 install pycocotools
!pip3 install -U scikit-image
!pip3 install -U cython 
#!pip3 install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI"

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

fasterRCNN_model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
in_features = fasterRCNN_model.roi_heads.box_predictor.cls_score.in_features
fasterRCNN_model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
class LossAverager:
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
day_sequence_1_dataset = LisaDataset(day_sequence_1_images, min_max_boxes_df_day1_BOX, True)
day_sequence_2_dataset = LisaDataset(day_sequence_2_images, min_max_boxes_df_day2_BOX, True)
    
# split the dataset in train and test set
#indices = torch.randperm(len(day_sequence_1_dataset)).tolist()
#day_sequence_1_dataset = torch.utils.data.Subset(day_sequence_1_dataset, indices[:-1000])
#day_sequence_1_dataset_val = torch.utils.data.Subset(day_sequence_1_dataset, indices[-1000:])

    # define training and validation data loaders
day_sequence_1_dataloader = torch.utils.data.DataLoader(
        day_sequence_1_dataset, batch_size=2, shuffle=True, num_workers=0,
        collate_fn = collate_fn)

day_sequence_2_dataloader = torch.utils.data.DataLoader(
        day_sequence_2_dataset, batch_size=1, shuffle=False, num_workers=0,
        collate_fn = collate_fn)

In [ ]:
from engine import train_one_epoch, evaluate
import time

train_loss_hist = LossAverager()
val_loss_hist = LossAverager()
EPOCHS = 10

fasterRCNN_model.double()
fasterRCNN_model.to(device)

params = [p for p in fasterRCNN_model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params)#, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)


for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")
    start_time = time.time()
    fasterRCNN_model.train()
    train_loss_hist.reset()
    
    for images, targets in tqdm(day_sequence_1_dataloader):
        
        images = torch.stack(images).to(device)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        bs = images.shape[0]
        
        loss_dict = fasterRCNN_model(images, targets)
        
        totalLoss = sum(loss for loss in loss_dict.values())
        lossValue = totalLoss.item()
        
        train_loss_hist.update(lossValue,bs)

        optimizer.zero_grad()
        totalLoss.backward()
        optimizer.step()
        lr_scheduler.step(totalLoss)
        
    print(f"Train loss: {train_loss_hist.avg}")
    evaluate(fasterRCNN_model, day_sequence_1_dataloader, device=device)